Consulta a un libro con retrieval augmented generation

CARGA DE TEXTO

In [25]:
import pymupdf
import pymupdf as fitz

# Ruta al archivo PDF
path = "C:\\Users\\ELI\\Downloads\\irbookonlinereading.pdf"
doc = pymupdf.open(path)
data=[]
for i, page in enumerate(doc):
    text = page.get_text().encode('utf-8')
    if text.strip():
        data.append({
            "page": i + 1,
            "raw": str(text).strip()
        })
# Convertir a DataFrame
import pandas as pd
df = pd.DataFrame(data)
df

,page,raw
0,1,b'Online edition (c)\n2009 Cambridge UP\nAn\nI...
1,2,b'Online edition (c)\n2009 Cambridge UP\n'
2,3,b'Online edition (c)\n2009 Cambridge UP\nAn\nI...
3,4,b'Online edition (c)\n2009 Cambridge UP\nDRAFT...
4,5,b'Online edition (c)\n2009 Cambridge UP\nDRAFT...
...,...,...
576,577,b'Online edition (c)\n2009 Cambridge UP\n540\n...
577,578,b'Online edition (c)\n2009 Cambridge UP\nIndex...
578,579,b'Online edition (c)\n2009 Cambridge UP\n542\n...
579,580,b'Online edition (c)\n2009 Cambridge UP\nIndex...


TABLA DE CONTENIDOS

In [26]:
# Verificar si el PDF tiene tabla de contenidos (TOC)
toc = doc.get_toc()

if toc:
    print(f"¡El PDF tiene {len(toc)} elementos en la tabla de contenidos!")
    print("\nPrimeras 10 entradas:")
    for i, (level, title, page) in enumerate(toc[:100]):
        indent = "  " * (level - 1)
        print(f"{indent}Nivel {level}: {title} (Página {page})")
else:
    print("El PDF no tiene tabla de contenidos estructurada")

¡El PDF tiene 256 elementos en la tabla de contenidos!

Primeras 10 entradas:
Nivel 1: List of Tables (Página 15)
Nivel 1: List of Figures (Página 19)
Nivel 1: Table of Notation (Página 27)
Nivel 1: Preface (Página 31)
Nivel 1: Boolean retrieval (Página 38)
  Nivel 2: An example information retrieval problem (Página 40)
  Nivel 2: A first take at building an inverted index (Página 43)
  Nivel 2: Processing Boolean queries (Página 47)
  Nivel 2: The extended Boolean model versus ranked retrieval (Página 51)
  Nivel 2: References and further reading (Página 54)
Nivel 1: The term vocabulary and postings lists (Página 56)
  Nivel 2: Document delineation and character sequence decoding (Página 56)
    Nivel 3: Obtaining the character sequence in a document (Página 56)
    Nivel 3: Choosing a document unit (Página 57)
  Nivel 2: Determining the vocabulary of terms (Página 59)
    Nivel 3: Tokenization (Página 59)
    Nivel 3: Dropping common terms: stop words (Página 64)
    Nivel 3: Normali

PREPROCESAR EL TEXTO

In [27]:
#preprocesar el texto
import re
#normalizar texto
def normalize_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r'\s+', ' ', text)  # Eliminar espacios en blanco extra
    text = re.sub(r'[^\w\s]', '', text)  # Eliminar caracteres especiales
    return text
# Aplicar normalización al DataFrame
df['normalized'] = df['raw'].apply(normalize_text)
# Verificar el resultado
df

,page,raw,normalized
0,1,b'Online edition (c)\n2009 Cambridge UP\nAn\nI...,bonline edition cn2009 cambridge upnannintrodu...
1,2,b'Online edition (c)\n2009 Cambridge UP\n',bonline edition cn2009 cambridge upn
2,3,b'Online edition (c)\n2009 Cambridge UP\nAn\nI...,bonline edition cn2009 cambridge upnannintrodu...
3,4,b'Online edition (c)\n2009 Cambridge UP\nDRAFT...,bonline edition cn2009 cambridge upndraftndo n...
4,5,b'Online edition (c)\n2009 Cambridge UP\nDRAFT...,bonline edition cn2009 cambridge upndraft xc2x...
...,...,...,...
576,577,b'Online edition (c)\n2009 Cambridge UP\n540\n...,bonline edition cn2009 cambridge upn540nindexn...
577,578,b'Online edition (c)\n2009 Cambridge UP\nIndex...,bonline edition cn2009 cambridge upnindexn541n...
578,579,b'Online edition (c)\n2009 Cambridge UP\n542\n...,bonline edition cn2009 cambridge upn542nindexn...
579,580,b'Online edition (c)\n2009 Cambridge UP\nIndex...,bonline edition cn2009 cambridge upnindexn543n...


In [28]:
#eliminar stopwords
from nltk.corpus import stopwords
# Descargar stopwords si no están disponibles
import nltk
nltk.download('stopwords')
# Definir stopwords en español
stop_words = set(stopwords.words('english'))
# Función para eliminar stopwords
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)
# Aplicar eliminación de stopwords al DataFrame
df['filtered'] = df['normalized'].apply(remove_stopwords)
# Verificar el resultado
df

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


,page,raw,normalized,filtered
0,1,b'Online edition (c)\n2009 Cambridge UP\nAn\nI...,bonline edition cn2009 cambridge upnannintrodu...,bonline edition cn2009 cambridge upnannintrodu...
1,2,b'Online edition (c)\n2009 Cambridge UP\n',bonline edition cn2009 cambridge upn,bonline edition cn2009 cambridge upn
2,3,b'Online edition (c)\n2009 Cambridge UP\nAn\nI...,bonline edition cn2009 cambridge upnannintrodu...,bonline edition cn2009 cambridge upnannintrodu...
3,4,b'Online edition (c)\n2009 Cambridge UP\nDRAFT...,bonline edition cn2009 cambridge upndraftndo n...,bonline edition cn2009 cambridge upndraftndo d...
4,5,b'Online edition (c)\n2009 Cambridge UP\nDRAFT...,bonline edition cn2009 cambridge upndraft xc2x...,bonline edition cn2009 cambridge upndraft xc2x...
...,...,...,...,...
576,577,b'Online edition (c)\n2009 Cambridge UP\n540\n...,bonline edition cn2009 cambridge upn540nindexn...,bonline edition cn2009 cambridge upn540nindexn...
577,578,b'Online edition (c)\n2009 Cambridge UP\nIndex...,bonline edition cn2009 cambridge upnindexn541n...,bonline edition cn2009 cambridge upnindexn541n...
578,579,b'Online edition (c)\n2009 Cambridge UP\n542\n...,bonline edition cn2009 cambridge upn542nindexn...,bonline edition cn2009 cambridge upn542nindexn...
579,580,b'Online edition (c)\n2009 Cambridge UP\nIndex...,bonline edition cn2009 cambridge upnindexn543n...,bonline edition cn2009 cambridge upnindexn543n...


EMBEDDING

In [29]:
from sentence_transformers import SentenceTransformer
import numpy as np


In [30]:
model = SentenceTransformer('all-MiniLM-L6-v2')
#generar funcion embeddings para todo el texto
def generate_embeddings(text):
    return model.encode(text)
# Aplicar la función de embeddings al DataFrame
df['embeddings'] = df['filtered'].apply(generate_embeddings)


ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /api/models/sentence-transformers/all-MiniLM-L6-v2/tree/main/additional_chat_templates?recursive=False&expand=False (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001BDF05D9350>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 848eec39-e852-4d2a-a5d4-27380a2d3685)')

In [ ]:
df

,page,raw,normalized,filtered,embeddings
0,1,b'Online edition (c)\n2009 Cambridge UP\nAn\nI...,bonline edition cn2009 cambridge upnannintrodu...,bonline edition cn2009 cambridge upnannintrodu...,"[-0.11581327, -0.04094364, 0.023199067, -0.036..."
1,2,b'Online edition (c)\n2009 Cambridge UP\n',bonline edition cn2009 cambridge upn,bonline edition cn2009 cambridge upn,"[-0.1323803, -0.023347827, 0.06720841, -0.0928..."
2,3,b'Online edition (c)\n2009 Cambridge UP\nAn\nI...,bonline edition cn2009 cambridge upnannintrodu...,bonline edition cn2009 cambridge upnannintrodu...,"[0.00705546, -0.048072208, 0.007882582, -0.068..."
3,4,b'Online edition (c)\n2009 Cambridge UP\nDRAFT...,bonline edition cn2009 cambridge upndraftndo n...,bonline edition cn2009 cambridge upndraftndo d...,"[-0.05006524, -0.0019235705, -0.016345194, -0...."
4,5,b'Online edition (c)\n2009 Cambridge UP\nDRAFT...,bonline edition cn2009 cambridge upndraft xc2x...,bonline edition cn2009 cambridge upndraft xc2x...,"[0.027868386, -0.087042965, -0.01009805, 0.016..."
...,...,...,...,...,...
576,577,b'Online edition (c)\n2009 Cambridge UP\n540\n...,bonline edition cn2009 cambridge upn540nindexn...,bonline edition cn2009 cambridge upn540nindexn...,"[-0.026038677, -0.12275319, -0.014319798, -0.0..."
577,578,b'Online edition (c)\n2009 Cambridge UP\nIndex...,bonline edition cn2009 cambridge upnindexn541n...,bonline edition cn2009 cambridge upnindexn541n...,"[0.02994777, -0.13199653, -0.008621623, -0.010..."
578,579,b'Online edition (c)\n2009 Cambridge UP\n542\n...,bonline edition cn2009 cambridge upn542nindexn...,bonline edition cn2009 cambridge upn542nindexn...,"[0.0034598007, -0.077709176, -0.0045174663, -0..."
579,580,b'Online edition (c)\n2009 Cambridge UP\nIndex...,bonline edition cn2009 cambridge upnindexn543n...,bonline edition cn2009 cambridge upnindexn543n...,"[-0.020311879, -0.13165753, 0.01688339, -0.020..."


In [ ]:
#sacar los embeddings del df
embeddings = np.array(df['embeddings'].tolist())
embeddings

array([[-0.11581327, -0.04094364,  0.02319907, ..., -0.0260279 ,
         0.02207126, -0.04298127],
       [-0.1323803 , -0.02334783,  0.06720841, ...,  0.01590319,
        -0.02216803,  0.01160852],
       [ 0.00705546, -0.04807221,  0.00788258, ...,  0.0255634 ,
        -0.02932286,  0.00203923],
       ...,
       [ 0.0034598 , -0.07770918, -0.00451747, ..., -0.01676263,
        -0.03796598, -0.0576323 ],
       [-0.02031188, -0.13165753,  0.01688339, ..., -0.06474322,
        -0.04680839, -0.02762038],
       [ 0.00927761, -0.12823053,  0.01517444, ..., -0.02842616,
        -0.03841518, -0.03689068]], dtype=float32)

QUERY

In [ ]:
#query
query = "stopwords"
query_embedding = model.encode(query)
query_embedding

array([-1.49560859e-03,  1.38110016e-02,  3.26872654e-02, -3.50735500e-03,
       -4.26756963e-03, -6.79441215e-03,  1.45399123e-01, -1.11046331e-02,
       -7.86744952e-02,  2.04429794e-02,  6.93996670e-03, -2.10056063e-02,
        4.66357656e-02, -4.85497192e-02, -7.59261772e-02,  8.11752751e-02,
        4.30341251e-02,  4.64086644e-02, -9.42633674e-02, -9.08575580e-03,
        6.45952672e-02,  1.34171650e-01,  2.02286150e-02,  2.76836660e-02,
       -1.09036537e-02,  5.80462441e-02, -9.04709026e-02, -6.54042652e-03,
        6.67781010e-02, -8.32427386e-03,  2.57875044e-02,  2.84907110e-02,
        6.08631112e-02,  2.46394724e-02,  2.32363176e-02, -3.44091505e-02,
       -1.05311200e-01,  5.39251119e-02,  2.46874914e-02, -1.99775379e-02,
       -7.40725473e-02, -1.03850476e-01, -1.05458960e-01, -4.92498577e-02,
        1.12935327e-01, -8.43069144e-03, -2.37405039e-02,  1.15957484e-02,
        5.11254892e-02,  2.55397037e-02, -7.89895132e-02, -5.44321947e-02,
       -2.77857948e-02,  

BASES DE DATOS VECTORIALES 

In [ ]:
import faiss
# Crear un índice FAISS para búsqueda de similitud
index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 distance
index.add(embeddings)  # Añadir los embeddings al índice


In [ ]:
# Realizar la búsqueda de similitud
k = 5  # Número de resultados a recuperar
D, I = index.search(np.array([query_embedding]), k)  # D: distancias, I: índices
#guardar en top_k_indices los índices de los k resultados más cercanos
top_k_indices = I[0]
top_k_indices

array([ 63, 275, 468,  69,  85], dtype=int64)

In [ ]:
# Mostrar los resultados de la búsqueda
for i, idx in enumerate(I[0]):
    print(f"Resultado {i + 1}:")
    print(f"Página: {df.iloc[idx]['page']}")
    print(f"Texto: {df.iloc[idx]['filtered'][:200]}...")  # Mostrar solo los primeros 200 caracteres
    print(f"Distancia: {D[0][i]}\n")



Resultado 1:
Página: 64
Texto: bonline edition cn2009 cambridge upn22ndetermining vocabulary termsn27n222ndropping common terms stop wordsnsometimes extremely common words would appear ofnlittle value helping select documents match...
Distancia: 0.981431245803833

Resultado 2:
Página: 276
Texto: bonline edition cn2009 cambridge upn121nlanguage modelsn239nmodel m1nmodel m2nthen02nthen015nan01nan012nfrogn001nfrogn00002ntoadn001ntoadn00001nsaidn003nsaidn003nlikesn002nlikesn004nthatn004nthatn004n...
Distancia: 1.1468493938446045

Resultado 3:
Página: 469
Texto: bonline edition cn2009 cambridge upn432n19nweb search basicsn194nthe search user experiencenit crucial understand users web search well isnagain signixefxacx81cant change traditional information retri...
Distancia: 1.1579183340072632

Resultado 4:
Página: 70
Texto: bonline edition cn2009 cambridge upn22ndetermining vocabulary termsn33nlinguistic processing stemming lemmatization often done adnditional plugin component indexing proce

In [ ]:
#unir los 5 resultados más similares
context = " ".join(df.iloc[index]['filtered'] for index in top_k_indices)
# Imprimir el contexto
print("Contexto de los resultados más similares:")
print(context)


Contexto de los resultados más similares:
bonline edition cn2009 cambridge upn22ndetermining vocabulary termsn27n222ndropping common terms stop wordsnsometimes extremely common words would appear ofnlittle value helping select documents matching user need excludednfrom vocabulary entirely words called stop words generalnstop wordsnstrategy determining stop list sort terms collection frequencyncollectionnfrequencynthe total number times term appears document collectionnand take frequent terms often handxefxacx81ltered senmantic content relative domain documents indexed asna stop list members discarded indexing annstop listnexample stop list shown figure 25 using stop list signixefxacx81cantlynreduces number postings system store presentnsome statistics chapter 5 see table 51 page 87 lot ofnthe time indexing stop words little harm keyword searches withnterms like donxe2x80x99t seem useful however true fornphrase searches phrase query xe2x80x9cpresident united statesxe2x80x9d conntains tw

INTEGRACIÓN CON IA

In [ ]:
prompt = f"""Eres una aplicación de Retrieval Augmented Generation que siempre responde en español. Usa el siguiente contexto para responder la pregunta.
Si la respuesta no está en el contexto, di que no sabes.

Contexto:
{context}

Pregunta:
El usuario está preguntando sobre: {query}
"""

In [ ]:
from openai import OpenAI

In [ ]:
client = OpenAI(api_key="")

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()


completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ]
)


#guardar el resultado en una variable
respuesta = completion.choices[0].message.content
respuesta

'Las "stop words" o palabras vacías son términos muy comunes en un idioma que a menudo se excluyen del procesamiento de texto porque tienen poco valor para la selección de documentos que coincidan con las necesidades del usuario. Estas palabras se descartan para optimizar el proceso de indexación y búsqueda, ya que su inclusión puede aumentar significativamente los costos de almacenamiento y procesamiento en los sistemas de recuperación de información (IR). Ejemplos de stop words son palabras como "y", "de", "la", que no aportan información significativa en búsquedas y se suelen eliminar de los índices para mejorar la eficiencia. Aunque esto puede no afectar mucho las búsquedas por palabras clave, en consultas de frases, la eliminación de stop words puede llevar a perder el significado completo de la búsqueda.'

TEXT TO SPEECH 

In [31]:
from gtts import gTTS
import io
import pygame

# Convertir texto a MP3 (en memoria)
tts = gTTS(text=respuesta, lang='es')
mp3_fp = io.BytesIO()
tts.write_to_fp(mp3_fp)
mp3_fp.seek(0)

# Inicializar pygame para reproducir
pygame.mixer.init()
pygame.mixer.music.load(mp3_fp)
pygame.mixer.music.play()

print("🎧 Reproduciendo audio...")
# Esperar a que termine la reproducción
while pygame.mixer.music.get_busy():
    pygame.time.Clock().tick(10)
#imprimir mensaje de finalización
print("✅ Reproducción finalizada.")


🎧 Reproduciendo audio...
✅ Reproducción finalizada.


TEXT TO SPEECH GEMINI

In [32]:
import requests
import json
import base64
import io
import pygame 


In [33]:
# Inicializar pygame mixer
pygame.mixer.init()

In [ ]:
API_KEY = ""
url = f"https://texttospeech.googleapis.com/v1/text:synthesize?key={API_KEY}"

In [ ]:
# Cuerpo de la solicitud
data = {
    "input": {"text": respuesta},
    "voice": {"languageCode": "es-ES", "ssmlGender": "NEUTRAL"},
    "audioConfig": {"audioEncoding": "MP3"}
}

# Enviar la solicitud a la API
response = requests.post(url, headers={"Content-Type": "application/json"}, data=json.dumps(data))

if response.status_code == 200:
    # Decodificar el contenido base64
    audio_base64 = response.json()["audioContent"]
    audio_data = base64.b64decode(audio_base64)

    # Cargar el audio en memoria usando un objeto BytesIO
    audio_stream = io.BytesIO(audio_data)

    # Cargar en pygame
    pygame.mixer.music.load(audio_stream)
    pygame.mixer.music.play()

    print("🎧 Reproduciendo audio...")

    # Esperar a que termine la reproducción
    while pygame.mixer.music.get_busy():
        continue
    #imprimir mensaje de finalización
    print("✅ Reproducción finalizada.")

else:
    print("❌ Error al obtener audio:", response.text)
    

🎧 Reproduciendo audio...
✅ Reproducción finalizada.
